# Probabilistic Programming

In this notebook we'll take a look at an approach for solving problems that have a great deal of *uncertainty*.  That is - we want to make use of the fact that we can quantify what we do and do not know.  We can write programs that take this uncertainty into account.  This is known as _probabilistic programming_.

The first step is to assume that there is a process that _generates_ the data (i.e., this is a generative model).  As such, this process has a _functional form_ and some _parameters_.  

Second, we apply *Bayes' Rule*:

$$ P(Parameters | Data) = \frac {P(Data | Parameters) \cdot P(Parameters)} {P(Data)} $$

In simpler terms:  In order to get values for the parameters (left-hand side), we need to:
*  Find the values of the parameters that are most _likely_ for the data we're seeing (left term in the numerator)
*  Assume how parameters are distributed (right term in the numerator)
*  Assume how data is distributed (denominator)

This can get very complicated very quickly, so we use _Monte-Carlo methods_:  These are statistical methods that rely on _random sampling_ of thousands or millions of values, and then seeing how likely these values are.  By combining these estimates we can then measure how certain/uncertain our results are.

# Motivating Example

Assume we have 3 speakers from our organization who present at a conference.  Attendees can indicate (maybe using a web app) whether they liked the talk or not.  Who is the best speaker?

Let's bring in the data!

In [ ]:
%pylab inline 
import pandas as pd     # Dataframe library
import pymc3 as pm      # Markov-Chain Monte-Carlo (MCMC) library
import arviz as az      # Visualization for MCMC libraries
import matplotlib       # Basic visualization library

# Default figure size
matplotlib.rcParams['figure.figsize'] = (20,7)

In [ ]:
from speakers import get_data

df = get_data()
df

In [ ]:
# Compute number of likes and unlikes

df_counts = df.groupby(['name', 'value']).size().unstack()
df_counts.columns = ['unlike','like']

df_counts

In [ ]:
# Add average value for each speaker

df_counts['total'] = df_counts['like'] + df_counts['unlike']
df_counts['avg'] = df_counts['like'] / df_counts['total']


df_counts

According to this, Yaniv is the best speaker while Yuval is the worst.  Is this a good way to analyse the data?  Why or why not?

## Math Detour

Assume we have some data, and would like to understand the process which created it.  That process has a *functional form* and some *parameters*.  This is called a *distribution*.  Let's explain.

### Statistical Distributions


A *statistical distribution* is simply a function that assigns a non-negative probability value to its inputs.  The sum of all values should be exactly equal to 1.0.  The inputs are known as a _sample space_ and can be anything we'd like.  

#### Example 1

Let's look at the well-known example of a coin toss.  The possible inputs - or sample space - are simply the sides of the coin.  The distribution will then assign each a value:

Heads --->   0.5

Tails --->   0.5

This is a fair coin, since the probability of both sides are equal.  If we toss the coin 100 times, we expect - on average - to get 50 heads and 50 tails.

#### Example 2

We can also have an _unfair_ (or biased) coin:

Heads ---> 0.2

Tails ---> 0.8

If we toss the coin 100 times we expect - on average - the coin to come up heads 20 times and tails 80 times.


### Bernoulli Distribution

In the previous examples we see a pattern:  sample space composed of two values where one has the probability $p$ and the other the probability $1-p$.  This is known as the _Bernoulli Distribution_ and has the functional form:

$$ P(X=x) = p^{x} \cdot (1-p)^{(1-x)} $$

Its intuitive explanation is as follows:  we run a single experiment which has two possible outputs.  We call one output '1' and the other '0'.  The probability for getting 1 is $p$ and the probability for getting 0 is $1-p$.

#### Example 3

Assume a regular 6-sided playing die.  Its sample space and associated values:

1 ---> 1/6

2 ---> 1/6

3 ---> 1/6

4 ---> 1/6

5 ---> 1/6

6 ---> 1/6


### Uniform Distribution 

When you have $n$ possible outcomes and each outcome is equally likely, you have a _uniform distribution_.  Its functional form:

$$ P(X=x) = \frac{1}{n} $$


## Back to our scheduled broadcast...

Using averages is probably not the best idea:
*  Different amounts of data for each speaker
*  Each speaker talks about a different subject - how can we compare?
*  Different people in each talk, with different interests and conditions - how can we compare?

Instead, let's try a different approach.  Assume each speaker has a _probability of getting a like_.  This includes:
*  Their ability to choose an interesting subject
*  Their ability to prepare a good presentation 
*  Their ability to deliver the presentation in an engaging manner,
*  More...

Why is it a probability?  Because there are different people that listen to each talk.  They may or may not like the presentation or delivery, may or may not be bothered by other things, etc.  Essentially, we are allowing for some _uncertainty_ to come into play.  

So we can the following:  The process that generates the likes for each speaker is based on n (the number of people) different 'experiments' where the chance of success is unique to each speaker.

In statistical terms:  Our data is Bernoulli-distributed, with a parameter $p$ that depends on the speaker.  We assume that any value of $p$ between 0 and 1 can be possible.

Let's write the code and find the value of $p$ that is most likely to have generated the data.



In [ ]:
# multiple speakers - we need to assign each observation ('experiment result') to a specific speaker
name_idx, names = pd.factorize(df['name'])
name_idx, names

In [ ]:
# The Model!!!

with pm.Model() as single_value:
    
    # The 'like probability' - between 0 and 1, assume there is a single value for all speakers
    like_probability = pm.Uniform("like_prob", lower=0.0, upper=1.0)
    
    # The generating process, with the prob. of likes and the actual data for each speaker
    likes = pm.Bernoulli("likes", p=like_probability, observed=df['value'][name_idx])
    
    # Sample 5000 points * 4 chains of possible like_probability values and see which describes the data better
    trace = pm.sample(5000, chains=4, return_inferencedata=False)
    
    # Summary statistics and plots
    print(az.summary(trace))
    az.plot_trace(trace)
    

Does the mean value of like_prob seem reasonable?  Why or why not?

## Another Math Detour (WHAT????)

### The Beta Distribution

You've probably come across the Bernoulli and Uniform distributions before.  And even if not - they're pretty intuitive.  Let's see another one, named the _Beta_ distribution.

First off, we won't show the functional form, as it's not really relevant.  What's important to understand is that in the Beta distribution, our sample space consists of all possible values between 0 and 1.  So there is a probability value for 0.0, 0.001, 0.393984, 0.2994774 and any other number between 0 and 1.  

This is interesting because it allows us to give a probability to our 'like_prob' variable!   HUH???

In our last model, we assumed nothing about the value of like_probability.  Or rather, we assumed all values are equally likely, from 0.0 to 1.0.  But is that right?  Conferences to bring in good speakers and people tend to go see speakers they're familiar with.  So an experienced speaker will likely have a like_probability in the upper range of 0 to 1.  We can use the Beta to model this!


In [ ]:
# Possible values of sample space
xs = np.arange(0,1.01,0.01)

# Generate a Beta that is centered on 0.8 and has a sd of 0.1
beta_dist = pm.Beta.dist(mu=0.8, sd=0.1)

# Plot
plot(xs, np.exp(beta_dist.logp(xs).eval()))

So now we have some better assumptions on how like_probility behaves.  This is called a _prior_.  We can simply plug it into the model.

In [ ]:
with pm.Model() as single_value_with_prior:
    
    # Like_probability is now assumed to have a Beta distribution prior
    like_probability = pm.Beta("like_prob", mu=0.8, sd=0.1)
    
    likes = pm.Bernoulli("likes", p=like_probability, observed=df['value'][name_idx])
    trace = pm.sample(5000, chains=4, return_inferencedata=False)
    
    print(az.summary(trace))
    az.plot_trace(trace)
    

Does this value make sense?  What about the fact that we have a single value for 3 speakers who may be talking about wildly different things?  Let's account for that in the model.

In [ ]:

# How many speakers do we have?
num_names = len(names)

with pm.Model() as multiple_values:
    
    # Still Beta-distributed, but now we have a 'shape' parameter with multiple such values
    like_probability = pm.Beta("like_prob", mu=0.8, sd=0.1, shape=num_names)
    
    # The Bernoulli parameter p now has an index that refers to the speaker-specific value
    likes = pm.Bernoulli("likes", p=like_probability[name_idx], observed=df['value'][name_idx])
    
    trace = pm.sample(5000, chains=4, return_inferencedata=False)
    
    print(az.summary(trace))
    az.plot_trace(trace)
    
    # Show a comparison of all speakers
    az.plot_forest([trace['like_prob'][:,i] for i in range(num_names)], combined=True, model_names=names)
    

We now have 3 values that are closer to what we assumed.  They are also more in-line with our experience as speakers.  Not only that, but it's no longer clear who the best speaker is, as all three are close.  


We still have another problem to deal with:  there are different amounts of data for each speaker.  That will depend on the subject of each talk - so speakers with less 'mainstream' interests will receive less results (likes or unlikes).  Is there something we can do?

Yes, there is!  In the previous model we assumed that the distribution of like_probability was identical for each speaker but independent - i.e., there is no relation between like_probability of one speaker to that of another.  But we know that all 3 speakers were trained by the same organization.  So maybe what we should do is parameterize the distribution of like_probability itself.  That is, the values for each speaker have their own distribution (or 'hyper-distribution').  This is known as a _hierarchical model_ and is extremely powerful.  In essence, what it allows us to do is to 'transfer' knowledge from a speaker with more attendees to one with less.


In [ ]:

with pm.Model() as hierarchical_multiple_values:
    
    # Assume that the average like_probability for all our speakers is in the range 0.65 to 0.85
    like_mean = pm.Uniform("like_mean", lower=0.65, upper=0.85)
    
    # Bring that assumption in
    like_probability = pm.Beta("like_prob", mu=like_mean, sd=0.05, shape=num_names)
    
    likes = pm.Bernoulli("likes", p=like_probability[name_idx], observed=df['value'][name_idx])
    
    trace = pm.sample(5000, return_inferencedata=False)
    
    print(az.summary(trace))
    az.plot_trace(trace)
    az.plot_forest([trace['like_prob'][:,i] for i in range(num_names)], combined=True, model_names=names)
    

So what about these values?

We can build bigger and bigger hierarchical models that will take more and more sources of knowledge into account.  But now it is time to compare our initial results to what we've accomplished.  Is the comparison clear?



In [ ]:
# Take the mean value like_prob for each speaker
df_counts['mcmc'] = np.array([trace['like_prob'][:,i].mean() for i in range(num_names)]).T

df_counts[['avg','mcmc']].plot()
df_counts

